In [1]:
# !pip install spacy # install spaCy
# !pip install tqdm # install tqdm package to display the progress
# !pip install textblob # for dictionary-based sentment analysis
# !pip install nrclex # for dictionary-based emotions analysis
# !pip install gensim # for word2vec and doc2vec 
!pip uninstall tensorflow -y
!pip install tensorflow==2.15.0 # install tensorflow for deep learning
!pip install keras # install keras for deep learning

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
  Obtaining dependency information for tensorflow==2.15.0 from https://files.pythonhosted.org/packages/93/c0/a774286d0383419f558deb27096e5de9f9facd6c27df8e9f9af6fba2f77e/tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.3 MB)


In [2]:
# Import other packages:

import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


import sklearn
from sklearn import tree
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

from gensim.models import Word2Vec

# tensorflow imports:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Normalization, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.utils import pad_sequences
from keras.optimizers import *
from keras.preprocessing import sequence

%matplotlib inline
plt.style.use('fivethirtyeight')

2024-07-18 02:15:49.297359: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 02:15:49.332317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 02:15:49.332349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 02:15:49.333240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 02:15:49.339401: I tensorflow/core/platform/cpu_feature_guar

In [3]:
!pip install bertopic

In [4]:
df = pd.read_csv("pm_wo_facts_having_txns.csv")

/tmp/ipykernel_1236/1947544649.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pm_wo_facts_having_txns.csv")


In [5]:
df.columns

Index(['workorderid', 'wostatus', 'wodesc', 'woproject', 'wotypedesc',
       'wodatecreated', 'wocategorywithdesc', 'wopropertyid',
       'wopropertydescription', 'wodatecompleted', 'cnt_wo_phases', 'cnt_pos',
       'cnt_lines', 'cnt_txns', 'amount_spent_labor', 'amount_spent_material'],
      dtype='object')

In [6]:
#Eliminate WOs w/ materials cost <= 0
df = df[df["amount_spent_material"] >0]

In [7]:
df.shape

(50920, 16)

In [8]:
#Taking out upper and lower 2.5% (boundaries from SPSS via Nick)
df = df[df["amount_spent_material"] >11]
df = df[df["amount_spent_material"] <5000]

In [9]:
df.shape

(48367, 16)

In [10]:
df['wodatecompleted']

14             NaN
33         6/10/14
82         6/14/14
101            NaN
117        6/12/14
            ...   
672805    12/11/18
672807     1/12/19
672818      1/5/19
672819    12/19/18
672824    12/24/19
Name: wodatecompleted, Length: 48367, dtype: object

In [11]:
df["cnt_txns"].fillna(0).describe()

count    48367.000000
mean        12.616784
std         92.213097
min          1.000000
25%          4.000000
50%          6.000000
75%         10.000000
max      11844.000000
Name: cnt_txns, dtype: float64

In [12]:
df["cnt_pos"].fillna(0).describe()

count    48367.000000
mean         0.765357
std          2.025169
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max        118.000000
Name: cnt_pos, dtype: float64

In [13]:
##Remove strings in parentheses (building codes/names)
import re

def remove_text_in_parentheses(text):
    # Use regex to find and replace text in parentheses
    return re.sub(r'\s*\(.*?\)', '', text)


In [14]:
##Remove numbers (building codes/numbers)

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [15]:
column_string = df['wodesc'].astype(str).str.upper()
clean_df= column_string.apply(remove_text_in_parentheses)
clean_df_final= clean_df.apply(remove_numbers)
clean_df_final_id = pd.DataFrame({
    'workorderid': df['workorderid'],
    'wodesc': clean_df_final
})

In [16]:
# import and load spaCy:

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy import displacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm') 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
# Add new stop words: intervals (monthly, weekly, annual, etc) and buildings' proper names (e.g, Rouss-Robertson)  

stop_words = pd.read_csv("Building name and time stop words.csv")

In [18]:
# Mark them as stop words
for w in stop_words['words']:
    nlp.vocab[w].is_stop = True

In [19]:
tqdm.pandas() # To display the progress
clean_df_final['pr_wodesc'] = clean_df_final.progress_apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop and token.is_alpha))

100%|██████████| 48367/48367 [04:05<00:00, 196.66it/s]


In [20]:
from bertopic import BERTopic

In [21]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(clean_df_final['pr_wodesc'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [22]:
topic_model.save("my_model", serialization="safetensors")

In [23]:
topic_model= BERTopic.load("my_model")

In [24]:
topic_model.topic_labels_

{-1: '-1_colonial_special_hazard_vsi',
 0: '0_system_suppression_inspection_fire',
 1: '1_generator_emergency_sci_eng',
 2: '2_visual_battery_detection_semi',
 3: '3_ahu_whse__',
 4: '4_detection_fire_times_alarm',
 5: '5_hydraulic_elevators_test_inspection',
 6: '6_lighting_intersession_list_paint',
 7: '7_detection_semi_fire_road',
 8: '8_interior_exterior_lighting_lamp',
 9: '9_general_plumbing_operate_room',
 10: '10_traction_elevators_test_inspection',
 11: '11_compressor_air_crane_inspect',
 12: '12_state_vehicle_inspection_',
 13: '13_emerg_signs_exit_lights',
 14: '14_backflow_preventer_fire_',
 15: '15_preventive_maintenance_vehicle_contract',
 16: '16_zone_maintenance_interior_',
 17: '17_annually_compressor_air_winter',
 18: '18_signs_exit_lights_emergency',
 19: '19_treatment_water_dual_temp',
 20: '20_fan_exhaust_fans_triannual',
 21: '21_traction_elevators_lifts_',
 22: '22_hydraulic_elevators_semi_inspection',
 23: '23_ahu_fcu_semi_graduation',
 24: '24_coil_unit_debris_

In [25]:
topic_small= topic_model.reduce_topics(clean_df_final['pr_wodesc'], nr_topics=30)

In [26]:
topic_small.topic_labels_

{-1: '-1_vehicle_ro_lof_colonial',
 0: '0_fire_suppression_system_detection',
 1: '1_elevators_test_hydraulic_traction',
 2: '2_generator_emergency_unit_coil',
 3: '3_lighting_lights_interior_exterior',
 4: '4_ahu_hrl_semi_bi',
 5: '5_air_compressor_pump_annually',
 6: '6_general_plumbing_treatment_water',
 7: '7_vehicle_ro_lof_rotate',
 8: '8_zone_maintenance_rooms_isolation',
 9: '9_exhaust_fan_heaters_cool',
 10: '10_door_room_follow_need',
 11: '11_boiler_inspector_department_vessel',
 12: '12_ice_machine_refrigerator_freezer',
 13: '13_filter_replacement_water_hvac',
 14: '14_pest_control_service_contract',
 15: '15_tank_gas_med_polish',
 16: '16_cleaning_window_sweeping_street',
 17: '17_split_system_glycol_semi',
 18: '18_grease_traps_turnover_lubricate',
 19: '19_termite_renewal_dodson_warranty',
 20: '20_boxes_vav_vfd_aed',
 21: '21_panelboard_years_year_dim',
 22: '22_wax_june_floor_december',
 23: '23_outlets_fittings_gas_power',
 24: '24_glove_voltage_rated_test',
 25: '25_

In [27]:
topic_small.topic_sizes_

Counter({0: 17016,
         1: 5784,
         2: 4737,
         3: 4216,
         4: 3263,
         5: 3181,
         6: 2234,
         7: 1769,
         8: 970,
         9: 912,
         10: 763,
         -1: 666,
         11: 515,
         12: 440,
         13: 409,
         14: 312,
         15: 252,
         16: 164,
         17: 163,
         18: 142,
         19: 105,
         20: 102,
         21: 101,
         22: 38,
         23: 35,
         24: 28,
         25: 15,
         26: 12,
         27: 12,
         28: 11})

In [28]:
topic_small.save("my_model_small", serialization="safetensors")

In [29]:
topic_model= BERTopic.load("my_model_small")

2024-07-18 02:27:12,130 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


In [30]:
topic_info_df = topic_small.get_topic_info()
topic_info_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,666,-1_vehicle_ro_lof_colonial,"[vehicle, ro, lof, colonial, vsi, auto, specia...",[vehicle state inspection VEHICLE COLONIAL AUT...
1,0,17016,0_fire_suppression_system_detection,"[fire, suppression, system, detection, visual,...","[FIRE SUPPRESSION SYSTEM inspection, FIRE SUPP..."
2,1,5784,1_elevators_test_hydraulic_traction,"[elevators, test, hydraulic, traction, inspect...","[ELEVATORS HYDRAULIC INSPECTION test, ELEVATOR..."
3,2,4737,2_generator_emergency_unit_coil,"[generator, emergency, unit, coil, recovery, e...","[EMERGENCY GENERATOR, EMERGENCY GENERATOR, EME..."
4,3,4216,3_lighting_lights_interior_exterior,"[lighting, lights, interior, exterior, exit, s...","[INTERIOR EXTERIOR LIGHTING, INTERIOR EXTERIOR..."
5,4,3263,4_ahu_hrl_semi_bi,"[ahu, hrl, semi, bi, ats, tri, fcu, bas, whse,...","[AHU, AHU, AHU]"
6,5,3181,5_air_compressor_pump_annually,"[air, compressor, pump, annually, pumps, vacuu...","[SUPPRESSION SYSTEM AIR COMPRESSOR, SUPPRESSIO..."
7,6,2234,6_general_plumbing_treatment_water,"[general, plumbing, treatment, water, drain, a...","[GENERAL PLUMBING, GENERAL PLUMBING, GENERAL P..."
8,7,1769,7_vehicle_ro_lof_rotate,"[vehicle, ro, lof, rotate, preventive, auto, m...",[VEHICLE PREVENTIVE maintenance VEHICLE MARSHA...
9,8,970,8_zone_maintenance_rooms_isolation,"[zone, maintenance, rooms, isolation, bldg, re...","[ZONE maintenance, ZONE MAINTENANCE, ZONE MAIN..."


In [31]:
header_names = ['workorderid','wodesc', 'Topic', 'prob']

# Create a DataFrame
df_concat = pd.DataFrame({
    header_names[0]: clean_df_final_id['workorderid'],
    header_names[1]: clean_df_final_id['wodesc'],
    header_names[2]: topic_small.topics_,
})

In [32]:
df_left_join = pd.merge(df_concat, topic_info_df, on='Topic', how='left')
df_left_join[['workorderid','wodesc','Topic','Name']]

,workorderid,wodesc,Topic,Name
0,1462943,FM SHOP # - MONTHLY PEST CONTROL SERVICE,14,14_pest_control_service_contract
1,1511593,EMERGENCY GENERATOR OIL TANK READINGS ZONE,2,2_generator_emergency_unit_coil
2,1512409,"LIGHT,EMERGENCY BATTERY CROSS ROADS -S BLDG.NO.",3,3_lighting_lights_interior_exterior
3,1247195,BAVARO HALL-MONTHLY PEST CONTROL SERVICE,14,14_pest_control_service_contract
4,1513539,"COMPRESSOR,MEDICAL,AIR MRL- ANNEX BLDG.NO. MER...",5,5_air_compressor_pump_annually
...,...,...,...,...
48362,2811249,MED GAS WEEKLY,15,15_tank_gas_med_polish
48363,2828739,AIR COMPRESSOR MONTHLY,5,5_air_compressor_pump_annually
48364,2815758,AIR COMPRESSOR MONTHLY,5,5_air_compressor_pump_annually
48365,2815888,WATER TREATMENT MONTHLY,6,6_general_plumbing_treatment_water


In [33]:
df_w_inventory_release = pd.read_csv("pm_wo_facts_having_txns (1).csv")
df_left_join = df_left_join[['workorderid','Topic',
 'Name']]
df_w_inventory_release["workorderid"]= df_w_inventory_release["workorderid"].astype(str)
df_left_join["workorderid"]= df_left_join["workorderid"].astype(str)

/tmp/ipykernel_1236/3361476914.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_w_inventory_release = pd.read_csv("pm_wo_facts_having_txns (1).csv")


In [34]:
df_combined = pd.merge(df_w_inventory_release, df_left_join, on='workorderid', how='inner')
df_combined[['workorderid',
 'wostatus',
 'wodesc',
 'woproject',
 'wotypedesc',
 'wodatecreated',
 'wocategorywithdesc',
 'wopropertyid',
 'wopropertydescription',
 'wodatecompleted',
 'cnt_wo_phases',
 'cnt_pos',
 'cnt_lines',
 'cnt_txns',
 'amount_spent_labor',
 'amount_spent_material',
'cnt_inv_releases',
'amount_material_inv_releases',
 'Topic',
 'Name']].to_csv("topics_pm_wo_facts_having_txns_inv_release_filtered.csv")

In [35]:
df_combined = pd.merge(df_w_inventory_release, df_left_join, on='workorderid', how='inner')
df_combined[['workorderid',
 'wostatus',
 'wodesc',
 'woproject',
 'wotypedesc',
 'wodatecreated',
 'wocategorywithdesc',
 'wopropertyid',
 'wopropertydescription',
 'wodatecompleted',
 'cnt_wo_phases',
 'cnt_pos',
 'cnt_lines',
 'cnt_txns',
 'amount_spent_labor',
 'amount_spent_material',
'cnt_inv_releases',
'amount_material_inv_releases',
 'Topic',
 'Name']].to_excel("xlsx_topics_pm_wo_facts_having_txns_inv_release_filtered.xlsx", index=False)

In [2]:
import pandas as pd

In [15]:
df = pd.read_excel("pm_wo_facts_having_txns (3).xlsx")

In [16]:
df.head()

,workorderid,wostatus,wodesc,woproject,wotypedesc,wodatecreated,wocategorywithdesc,wopropertyid,wopropertydescription,wodatecompleted,cnt_wo_phases,cnt_pos,cnt_lines,cnt_txns,amount_spent_labor,amount_spent_material,cnt_inv_releases,amount_material_inv_releases,max_labor_complete_date
0,2248311,CLOSED,SEWAGE PUMPS WEEKLY (FONTANA FOOD CENTER),NaN,Maintenance services including utility systems...,2015-01-22,PM (Routine/Proactive/Preventive work orders i...,0595,FONTANA FOOD CENTER,2015-04-18,3,NaN,NaN,6,86.43,0.0,0,0.0,2015-02-06 13:07:48.303
1,2961628,CLOSED,EMERGENCY LIGHTS/EXIT SIGNS MONTHLY (PT:A) (BI...,NaN,Maintenance services including utility systems...,2019-09-05,PM CODE (Code related Preventive maintenance w...,2146,BICE HOUSE,2019-12-12,1,NaN,NaN,2,28.53,0.0,0,0.0,2019-10-02 12:24:55.000
2,3051585,CLOSED,DEIONIZED WATER SYS WEEKLY (MEDICAL RESEARCH L...,NaN,Maintenance services including utility systems...,2020-04-09,PM (Routine/Proactive/Preventive work orders i...,1157,MEDICAL RESEARCH LAB (MR-4),2020-06-25,1,NaN,NaN,2,114.11,0.0,0,0.0,2020-04-15 14:26:37.000
3,1543349,CLOSED,"ICE MACHINE,BIO ENG MR-5 1155 (LIST)",NaN,Maintenance services including utility systems...,2014-12-25,PM (Routine/Proactive/Preventive work orders i...,1155,BIOMED. ENG. & MEDICAL SCI. BLD. (MR-5),2015-05-04,1,NaN,NaN,3,335.19,0.0,0,0.0,2015-02-23 14:12:40.767
4,3476959,CLOSED,HRL WATER TREATMENT LTHW - MONTHLY SEASONAL (S...,4519,Maintenance services including utility systems...,2023-01-26,PM (Routine/Proactive/Preventive work orders i...,2817,Copeley Apts #17 (324 Peyton Court),2023-05-05,1,NaN,NaN,2,204.78,0.0,0,0.0,2023-02-24 06:18:10.573


In [17]:
df2 = pd.read_csv("topics_pm_wo_facts_having_txns_inv_release_filtered.csv")

In [19]:
df2.columns

Index(['Unnamed: 0', 'workorderid', 'wostatus', 'wodesc', 'woproject',
       'wotypedesc', 'wodatecreated', 'wocategorywithdesc', 'wopropertyid',
       'wopropertydescription', 'wodatecompleted', 'cnt_wo_phases', 'cnt_pos',
       'cnt_lines', 'cnt_txns', 'amount_spent_labor', 'amount_spent_material',
       'cnt_inv_releases', 'amount_material_inv_releases', 'Topic', 'Name'],
      dtype='object')

In [18]:
df= df[['workorderid','max_labor_complete_date']]

In [20]:
df_combined = pd.merge(df2, df, on='workorderid', how='inner')
df_combined[['workorderid',
 'wostatus',
 'wodesc',
 'woproject',
 'wotypedesc',
 'wodatecreated',
 'wocategorywithdesc',
 'wopropertyid',
 'wopropertydescription',
 'wodatecompleted',
 'cnt_wo_phases',
 'cnt_pos',
 'cnt_lines',
 'cnt_txns',
 'amount_spent_labor',
 'amount_spent_material',
'cnt_inv_releases',
'amount_material_inv_releases',
 'Topic',
 'Name',
'max_labor_complete_date']].to_excel("xlsx_topics_pm_wo_facts_having_txns_inv_release_filtered.xlsx", index=False)

In [21]:
df_combined = pd.merge(df2, df, on='workorderid', how='inner')
df_combined[['workorderid',
 'wostatus',
 'wodesc',
 'woproject',
 'wotypedesc',
 'wodatecreated',
 'wocategorywithdesc',
 'wopropertyid',
 'wopropertydescription',
 'wodatecompleted',
 'cnt_wo_phases',
 'cnt_pos',
 'cnt_lines',
 'cnt_txns',
 'amount_spent_labor',
 'amount_spent_material',
'cnt_inv_releases',
'amount_material_inv_releases',
 'Topic',
 'Name',
'max_labor_complete_date']].to_excel("xlsx_topics_pm_wo_facts_having_txns_inv_release_filtered.xlsx", index=False)